In [1]:
from vllm import LLM, SamplingParams
from tqdm import tqdm
import datasets
import json
import argparse 

parser = argparse.ArgumentParser(description='Set model name.')  
# parser.add_argument('--model-name', type=str, required=True, help='Name of the model to use')  
# parser.add_argument('--model-path', type=str, default="/home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface", help='Dir of the model to use') 
# args = parser.parse_args()  
# path_dir = args.model_path
# model_name = args.model_name 

# template = "{{ '<|begin_of_text|>' }}{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ '<|start_header_id|>system<|end_header_id|>\n\n' + system_message + '<|eot_id|>' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ '<|start_header_id|>user<|end_header_id|>\n\n' + content + '<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n' }}{% elif message['role'] == 'assistant' %}{{ content + '<|eot_id|>' }}{% endif %}{% endfor %}"
# template = "{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ system_message + '\n' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ 'Human: ' + content + '\nAssistant:' }}{% elif message['role'] == 'assistant' %}{{ content + '<|end_of_text|>' + '\n' }}{% endif %}{% endfor %}"
model_name = "Phi-3-small-8k-instruct"
path_dir = "/mnt/lingjiejiang/textual_aesthetics/model_checkpoint/sft_merge_checkpoints/Phi-3-small-8k-instruct"
# path_dir = "Phi-3-small-8k-instruct"
# model_name = "Meta-Llama-3.1-8B-Instruct"
# model_name = "Meta-Llama-3.1-8B"
# Create an LLM.
llm = LLM(model=f"{path_dir}", trust_remote_code=True)

print(f"model name: {model_name}")
print(f"model_path: {path_dir}/{model_name}")

gen_kwargs_vllm = {
    "max_tokens": 2048,
    "temperature": 0.0,
}
tokenizer = llm.get_tokenizer()
if tokenizer.chat_template is None:
    tokenizer.chat_template = template
    # tokenizer.chat_template = tokenizer.chat_template.replace("<|eot_id|>", tokenizer.eos_token)
    # tokenizer.chat_template
    gen_kwargs_vllm['stop_token_ids'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]
    print(f"tokenizer.chat_template: {tokenizer.chat_template}")
    print("tokenizer is None, use setted template")
else:
    gen_kwargs_vllm['stop_token_ids'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|end_of_text|>")]
    print("use original template")
messages = tokenizer.apply_chat_template(messages, tokenize=False)


sampling_params = SamplingParams(**gen_kwargs_vllm)

eval_set = datasets.load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")["eval"]

def convert_to_message(example):  
    messages = [{"role": "user", "content": example["instruction"]}]  
    example["messages"] = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  
    return example  
eval_set = eval_set.map(convert_to_message)

encoded_inputs = tokenizer.batch_encode_plus(  
    eval_set['messages'],  
    add_special_tokens=False,
) 
input_ids = encoded_inputs['input_ids']  

outputs = llm.generate(prompt_token_ids=input_ids, sampling_params=sampling_params)
outputs_text = [x.outputs[0].text for x in outputs]
eval_set = eval_set.remove_columns(["output"])  # Remove the existing 'output' column if it exists  
eval_set = eval_set.remove_columns(["messages"])
eval_set = eval_set.add_column("output", outputs_text)  
def rename_generator(sample):
    sample['generator'] = f"{model_name}"
    return sample
eval_set = eval_set.map(rename_generator)
eval_set.to_json(f"{model_name}.jsonl", batch_size=128, num_proc=8)

## save data

export_dataset = eval_set
export_data_list = [dict(row) for row in export_dataset]
print(f"export data length {len(export_data_list)}")
# with open(f'./data/{model_name}.json', 'w', encoding='utf-8') as f:  
#     json.dump(export_data_list, f, ensure_ascii=False, indent=4)  

/home/v-lingjiang/miniconda3/envs/eval/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-13 13:29:06,453	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 11-13 13:29:09 llm_engine.py:174] Initializing an LLM engine (v0.5.4) with config: model='/mnt/lingjiejiang/textual_aesthetics/model_checkpoint/sft_merge_checkpoints/Phi-3-small-8k-instruct', speculative_config=None, tokenizer='/mnt/lingjiejiang/textual_aesthetics/model_checkpoint/sft_merge_checkpoints/Phi-3-small-8k-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=/mnt/lingjiejiang/textual_aesthetics/model_checkpoint/sft_merge_checkpoi

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:06<00:18,  6.10s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:12<00:12,  6.13s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:18<00:06,  6.34s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:19<00:00,  4.08s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:19<00:00,  4.86s/it]


INFO 11-13 13:29:29 model_runner.py:732] Loading model weights took 14.5384 GB


INFO 11-13 13:29:30 gpu_executor.py:102] # GPU blocks: 28181, # CPU blocks: 2048
INFO 11-13 13:29:33 model_runner.py:1024] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-13 13:29:33 model_runner.py:1028] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 11-13 13:29:43 model_runner.py:1225] Graph capturing finished in 10 secs.
model name: Phi-3-small-8k-instruct
model_path: /mnt/lingjiejiang/textual_aesthetics/model_checkpoint/sft_merge_checkpoints/Phi-3-small-8k-instruct/Phi-3-small-8k-instruct
use original template


NameError: name 'messages' is not defined

In [ ]:
import transformers  
import torch  
from tqdm import tqdm  
import datasets  
import json  
import argparse  
  
# Argument parsing  
parser = argparse.ArgumentParser(description='Set model name.')    
parser.add_argument('--model-name', type=str, required=True, help='Name of the model to use')    
parser.add_argument('--model-path', type=str, default="/home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface", help='Dir of the model to use')   
args = parser.parse_args()    
  
path_dir = args.model_path  
model_name = args.model_name   
  
# Load the tokenizer and model  
tokenizer = transformers.AutoTokenizer.from_pretrained(path_dir, trust_remote_code=True)  
model = transformers.AutoModelForCausalLM.from_pretrained(path_dir, trust_remote_code=True, torch_dtype=torch.bfloat16)  
model.to("cuda")  
  
print(f"model name: {model_name}")  
print(f"model_path: {path_dir}/{model_name}")  
  
gen_kwargs_hf = {  
    "max_new_tokens": 2048,  
    "temperature": 0.0,  
    "do_sample": False,  
}  
  
def generate_response(prompt):  
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)  
    tokens = model.generate(**inputs, **gen_kwargs_hf)  
    prompt = prompt.replace("<|begin_of_text|>", "")
    return tokenizer.decode(tokens[0], skip_special_tokens=True).replace(prompt, '')  
  
# Load dataset  
eval_set = datasets.load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")["eval"]  
  
# Convert dataset examples to messages  
def convert_to_message(example):  
    messages = [{"role": "user", "content": example["instruction"]}]  
    example["messages"] = tokenizer.convert_tokens_to_string(tokenizer.tokenize(tokenizer.chat_template.format(messages=messages)))  
    return example    
  
eval_set = eval_set.map(convert_to_message)  
  
# Generate responses  
outputs_text = []  
for example in tqdm(eval_set):  
    response = generate_response(example['messages'])  
    outputs_text.append(response)  
  
# Update dataset with model outputs  
eval_set = eval_set.remove_columns(["output"])  # Remove the existing 'output' column if it exists    
eval_set = eval_set.remove_columns(["messages"])  
eval_set = eval_set.add_column("output", outputs_text)  
  
def rename_generator(sample):  
    sample['generator'] = f"{model_name}"  
    return sample  
  
eval_set = eval_set.map(rename_generator)  
eval_set.to_json(f"{model_name}.jsonl", batch_size=128, num_proc=8)  
  
# Save data  
export_dataset = eval_set  
export_data_list = [dict(row) for row in export_dataset]  
print(f"export data length {len(export_data_list)}")  
with open(f'./data/{model_name}.json', 'w', encoding='utf-8') as f:  
    json.dump(export_data_list, f, ensure_ascii=False, indent=4)  

In [ ]:
import transformers
import torch
import datasets  
import json  
device = "cuda"

# Load the tokenizer and model
tokenizer = transformers.AutoTokenizer.from_pretrained("/mnt/lingjiejiang/textual_aesthetics/exp/saves/bitnet_glan2_2048_default_template/fullft_lr2e5_4kstep/sft/checkpoint-3000", trust_remote_code=True)
model = transformers.AutoModelForCausalLM.from_pretrained("/mnt/lingjiejiang/textual_aesthetics/exp/saves/bitnet_glan2_2048_default_template/fullft_lr2e5_4kstep/sft/checkpoint-3000", trust_remote_code=True, torch_dtype=torch.bfloat16, attn_implementation = "flash_attention_2")
model.to(device)
gen_kwargs_hf = {  
    "max_new_tokens": 2048,  
    "temperature": 0.0,  
    "do_sample": False,  
}  
  
def generate_response(prompt):  
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)  
    tokens = model.generate(**inputs, **gen_kwargs_hf)  
    return tokenizer.decode(tokens[0], skip_special_tokens=True).replace(prompt, '')  
  
# Load dataset  
eval_set = datasets.load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")["eval"]  
  
# Convert dataset examples to messages  
def convert_to_message(example):  
    messages = [{"role": "user", "content": example["instruction"]}]  
    example["messages"] = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  
    return example  
   
  
eval_set = eval_set.map(convert_to_message)  
  

Map:   0%|          | 0/805 [00:00<?, ? examples/s]


KeyError: '% if messages'

In [17]:
gen_kwargs_hf = {  
    "max_new_tokens": 2048,  
    "temperature": 0.0,  
    "do_sample": False,  
}  
  
def generate_response(prompt):  
    inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)  
    print(inputs)
    # print(tokenizer.decode(inputs))
    # 从inputs中获取input_ids  
    input_ids = inputs["input_ids"]  
    
    # 将input_ids解码回字符串  
    decoded_string = tokenizer.decode(input_ids[0])  
    
    print(decoded_string)
    return ""
    # tokens = model.generate(**inputs, **gen_kwargs_hf)  
    # return tokenizer.decode(tokens[0], skip_special_tokens=True).replace(prompt, '')  
  
# Load dataset  
eval_set = datasets.load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")["eval"]  
  
# Convert dataset examples to messages  
def convert_to_message(example):  
    messages = [{"role": "user", "content": example["instruction"]}]  
    example["messages"] = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  
    return example  
  
eval_set = eval_set.map(convert_to_message)  
  
# Generate responses  
outputs_text = []  
for example in eval_set.select(range(2)):  
    response = generate_response(example['messages'])  
    outputs_text.append(response)  

{'input_ids': tensor([[128000,  35075,     25,   3639,    527,    279,   5144,    315,   1063,
          11495,  20142,    430,   3940,    872,  31133,    389,  37776,   5380,
          72803,     25]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}
<|begin_of_text|>Human: What are the names of some famous actors that started their careers on Broadway?
Assistant:
{'input_ids': tensor([[128000,  35075,     25,   2650,   1550,   2326,   5415,    636,    872,
           5144,   5380,  72803,     25]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
<|begin_of_text|>Human: How did US states get their names?
Assistant:


In [20]:
# print(f"model name: {model_name}")  
# print(f"model_path: {path_dir}/{model_name}")  
  
from tqdm import tqdm
gen_kwargs_hf = {  
    "max_new_tokens": 2048,  
    "temperature": 0.0,  
    "do_sample": False,  
}  
  
def generate_responses(prompts):  
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True, add_special_tokens=False).to(model.device)  
    print(inputs)
    tokens = model.generate(**inputs, **gen_kwargs_hf)  
    return [tokenizer.decode(t, skip_special_tokens=True).replace(prompt, '') for t, prompt in zip(tokens, prompts)]  
  
# Load dataset  
eval_set = datasets.load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")["eval"]  
  
# Convert dataset examples to messages  
def convert_to_message(example):  
    messages = [{"role": "user", "content": example["instruction"]}]  
    example["messages"] = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  
    return example  
  
eval_set = eval_set.map(convert_to_message)  
eval_set = eval_set.select(range(5))
# Generate responses in batches  
batch_size = 32  
outputs_text = []  
for i in tqdm(range(0, len(eval_set), batch_size)):  
    batch_messages = eval_set[i:i+batch_size]['messages']  
    responses = generate_responses(batch_messages)  
    outputs_text.extend(responses)  

  0%|          | 0/1 [00:00<?, ?it/s]/home/v-lingjiang/miniconda3/envs/eval/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


{'input_ids': tensor([[128000,  35075,     25,   3639,    527,    279,   5144,    315,   1063,
          11495,  20142,    430,   3940,    872,  31133,    389,  37776,   5380,
          72803,     25, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
         128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
         128001, 128001, 128001],
        [128000,  35075,     25,   2650,   1550,   2326,   5415,    636,    872,
           5144,   5380,  72803,     25, 128001, 128001, 128001, 128001, 128001,
         128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
         128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
         128001, 128001, 128001],
        [128000,  35075,     25,  21694,     11,    856,  13219,    323,   1077,
          85612,   1390,    757,    311,   1514,  10536,   4047,    449,   1124,
             13,   3053,    499,  10552,   1268,    279,   1847,    374,   6476,
             11,    779,   

100%|██████████| 1/1 [03:21<00:00, 201.97s/it]


In [ ]:
import transformers  
import torch  
from tqdm import tqdm  
import datasets  
import json  
import argparse  
  
# Argument parsing  
parser = argparse.ArgumentParser(description='Set model name.')  
parser.add_argument('--model-name', type=str, required=True, help='Name of the model to use')  
parser.add_argument('--model-path', type=str, default="/home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface", help='Dir of the model to use')  
args = parser.parse_args()  
  
path_dir = args.model_path  
model_name = args.model_name  
  
# Load the tokenizer and model  
tokenizer = transformers.AutoTokenizer.from_pretrained(path_dir, trust_remote_code=True)  
model = transformers.AutoModelForCausalLM.from_pretrained(path_dir, trust_remote_code=True, torch_dtype=torch.bfloat16)  
model.to("cuda")  
  
print(f"model name: {model_name}")  
print(f"model_path: {path_dir}/{model_name}")  
  
gen_kwargs_hf = {  
    "max_new_tokens": 2048,  
    "temperature": 0.0,  
    "do_sample": False,  
}  
  
gen_kwargs_hf['eos_token_id'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|end_of_text|>")]
gen_kwargs_hf['pad_token_id'] = tokenizer.eos_token_id 

def generate_response(prompt):  
    inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)  
    tokens = model.generate(**inputs, **gen_kwargs_hf)  
    prompt = prompt.replace("<|begin_of_text|>", "")
    return tokenizer.decode(tokens[0], skip_special_tokens=True).replace(prompt, '')  
  
# Load dataset  
eval_set = datasets.load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")["eval"]  
  
# Convert dataset examples to messages  
def convert_to_message(example):  
    messages = [{"role": "user", "content": example["instruction"]}]  
    example["messages"] = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  
    return example  
  
eval_set = eval_set.map(convert_to_message)  
  
# Generate responses  
outputs_text = []  
for example in tqdm(eval_set):  
    response = generate_response(example['messages'])  
    outputs_text.append(response)  
  
# Update dataset with model outputs  
eval_set = eval_set.remove_columns(["output"])  # Remove the existing 'output' column if it exists  
eval_set = eval_set.remove_columns(["messages"])  
eval_set = eval_set.add_column("output", outputs_text)  
  
def rename_generator(sample):  
    sample['generator'] = f"{model_name}"  
    return sample  
  
eval_set = eval_set.map(rename_generator)  
eval_set.to_json(f"{model_name}.jsonl", batch_size=128, num_proc=8)  
  
# Save data  
export_dataset = eval_set  
export_data_list = [dict(row) for row in export_dataset]  
print(f"export data length {len(export_data_list)}")  
with open(f'./data/{model_name}.json', 'w', encoding='utf-8') as f:  
    json.dump(export_data_list, f, ensure_ascii=False, indent=4)  

'<|begin_of_text|>Human: hello\nAssistant:'

In [ ]:
  
gen_kwargs_hf = {  
    "max_new_tokens": 2048,  
    "temperature": 0.0,  
    "do_sample": False,  
}  
  
gen_kwargs_hf['eos_token_id'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|end_of_text|>")]
gen_kwargs_hf['pad_token_id'] = tokenizer.eos_token_id 

def generate_response(prompt):  
    inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)  
    tokens = model.generate(**inputs, **gen_kwargs_hf)  
    input = prompt.replace("<|begin_of_text|>", "")
    return tokenizer.decode(tokens[0], skip_special_tokens=True).replace(prompt, '')  
  
# Load dataset  
eval_set = datasets.load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")["eval"]  
  
# Convert dataset examples to messages  
def convert_to_message(example):  
    messages = [{"role": "user", "content": example["instruction"]}]  
    example["messages"] = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  
    return example  
  
eval_set = eval_set.map(convert_to_message)  
  
# Generate responses  
outputs_text = []  
for example in tqdm(eval_set):  
    response = generate_response(example['messages'])  
    outputs_text.append(response)  

  0%|          | 0/805 [00:00<?, ?it/s]/home/v-lingjiang/miniconda3/envs/eval/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
  0%|          | 3/805 [03:25<15:40:41, 70.38s/it]

In [ ]:
import transformers
import torch
import datasets  
import json  
from tqdm import tqdm
device = "cuda"

# Load the tokenizer and model
tokenizer = transformers.AutoTokenizer.from_pretrained("/mnt/lingjiejiang/textual_aesthetics/exp/saves/bitnet_glanchat_v2.1_8b_2048_default_template/fullft_lr5e6_e3/sft/checkpoint-3500", trust_remote_code=True)
model = transformers.AutoModelForCausalLM.from_pretrained("/mnt/lingjiejiang/textual_aesthetics/exp/saves/bitnet_glanchat_v2.1_8b_2048_default_template/fullft_lr5e6_e3/sft/checkpoint-3500", trust_remote_code=True, torch_dtype=torch.bfloat16, attn_implementation = "flash_attention_2")
model.to(device)

  
# gen_kwargs_hf = {  
#     "max_new_tokens": 2048,  
#     "temperature": 0.7,
#     "top_p":0.95,  
#     "do_sample": True,  
# }  
  
# gen_kwargs_hf['eos_token_id'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|end_of_text|>")]
# gen_kwargs_hf['pad_token_id'] = tokenizer.eos_token_id 

# def generate_response(prompt):  
#     print(f"prompt: {prompt}")
#     inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)  
#     print(f"inputs:{inputs}")
#     tokens = model.generate(**inputs, **gen_kwargs_hf)  
#     res = tokenizer.decode(tokens[0], skip_special_tokens=True)
#     print(res)
#     return tokenizer.decode(tokens[0], skip_special_tokens=True).replace(prompt, '')  
  
# # Load dataset  
# eval_set = datasets.load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")["eval"]  
  
# # Convert dataset examples to messages  
# def convert_to_message(example):  
#     messages = [{"role": "user", "content": example["instruction"]}]  
#     example["messages"] = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  
#     return example  
  
# eval_set = eval_set.map(convert_to_message)  
  
# # Generate responses  
# outputs_text = []  
# for example in tqdm(eval_set):  
#     response = generate_response(example['messages'])  
#     print(response)
#     outputs_text.append(response)  

/home/v-lingjiang/miniconda3/envs/eval/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [ ]:
gen_kwargs_hf = {  
    "max_new_tokens": 2048,  
    "temperature": 0.0,  
    "do_sample": False,  
}  
  
gen_kwargs_hf['eos_token_id'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|end_of_text|>")]
gen_kwargs_hf['pad_token_id'] = tokenizer.eos_token_id 

def generate_response(prompt):  
    print(f"prompt: {prompt}")
    inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)  
    print(f"inputs:{inputs}")
    tokens = model.generate(**inputs, **gen_kwargs_hf)  
    res = tokenizer.decode(tokens[0], skip_special_tokens=True)
    print(f"res: {res}")
    res_raw = tokenizer.decode(tokens[0], skip_special_tokens=False)
    print(f"res_raw: {res_raw}")    
    input = prompt.replace("<|begin_of_text|>", "")
    return tokenizer.decode(tokens[0], skip_special_tokens=True).replace(input, '')  
  
# Load dataset  
eval_set = datasets.load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")["eval"]  
  
# Convert dataset examples to messages  
def convert_to_message(example):  
    messages = [{"role": "user", "content": example["instruction"]}]  
    example["messages"] = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  
    return example  
  
eval_set = eval_set.map(convert_to_message)  
  
# Generate responses  
outputs_text = []  
for example in tqdm(eval_set):  
    response = generate_response(example['messages'])  
    print(response)
    outputs_text.append(response)  

  0%|          | 0/805 [00:00<?, ?it/s]/home/v-lingjiang/miniconda3/envs/eval/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


prompt: <|begin_of_text|>Human: What are the names of some famous actors that started their careers on Broadway?
Assistant:
inputs:{'input_ids': tensor([[128000,  35075,     25,   3639,    527,    279,   5144,    315,   1063,
          11495,  20142,    430,   3940,    872,  31133,    389,  37776,   5380,
          72803,     25]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


  0%|          | 1/805 [00:08<1:52:52,  8.42s/it]

res: Human: What are the names of some famous actors that started their careers on Broadway?
Assistant:Some famous actors who started their careers on Broadway include:

1. **Michael Jackson**: Known for his iconic dance moves and singing abilities.
2. **Michael Jackson**: Known for his iconic dance moves and singing abilities.
3. **Michael Jackson**: Known for his iconic dance moves and singing abilities.
4. **Michael Jackson**: Known for his iconic dance moves and singing abilities.
5. **Michael Jackson**: Known for his iconic dance moves and singing abilities.

These actors are just a few examples, but there are many more who have made significant contributions to the world of theater.
res_raw: <|begin_of_text|>Human: What are the names of some famous actors that started their careers on Broadway?
Assistant:Some famous actors who started their careers on Broadway include:

1. **Michael Jackson**: Known for his iconic dance moves and singing abilities.
2. **Michael Jackson**: Known f

  0%|          | 2/805 [02:36<20:13:51, 90.70s/it]

res: Human: How did US states get their names?
Assistant:The names of US states are derived from various sources, including Native American tribes, European explorers, and even the names of the original thirteen colonies. Here are a few examples:

1. **Native American Tribes**: Many states are named after Native American tribes that lived in the area. For example, California is named after the Chumash tribe, and Arizona is named after the Apache tribe.

2. **European Explorers**: Some states were named after explorers who discovered the area. For example, Florida was named after the Spanish explorer Juan Ponce de León, and Texas was named after the French explorer René de Laudre.

3. **Original Thirteen Colonies**: Some states were named after the original thirteen colonies that were established in the 17th and 18th centuries. For example, Virginia was named after the English explorer John Smith, and New York was named after the Dutch explorer Henry Hudson.

4. **State Names**: Some st

  0%|          | 3/805 [03:16<15:04:13, 67.65s/it]

res: Human: Hi, my sister and her girlfriends want me to play kickball with them. Can you explain how the game is played, so they don't take advantage of me?
Assistant:Sure, I'd be happy to help! Kickball is a fun game that involves kicking a ball to score points. Here's a step-by-step guide to how the game is played:

### 1. Equipment
- **Ball**: A soft, round ball is used in the game. It can be made of various materials, but a common choice is a rubber ball.
- **Field**: A rectangular field is used as the playing surface. The field can be made of grass, sand, or any other material.
- **Goalposts**: Two goalposts are placed at each end of the field. These posts serve as the scoring points.

### 2. Rules
- **Objective**: The goal of the game is to score as many points as possible within a given time frame.
- **Scoring**: Points are scored by kicking the ball into the goalposts. Each point is worth 1 point.
- **Time Frame**: The game is played within a specified time frame, usually 10 m

In [9]:
eval_set[0]['messages'].replace("<|begin_of_text|>", "")

'Human: What are the names of some famous actors that started their careers on Broadway?\nAssistant:'

In [1]:
import transformers
import torch
import datasets  
import json  
from tqdm import tqdm
device = "cuda"

# Load the tokenizer and model
tokenizer = transformers.AutoTokenizer.from_pretrained("/mnt/lingjiejiang/textual_aesthetics/exp/saves/bitnet_glan2_2048_default_template/fullft_lr2e5_4kstep/sft/checkpoint-3000", trust_remote_code=True)
model = transformers.AutoModelForCausalLM.from_pretrained("/mnt/lingjiejiang/textual_aesthetics/exp/saves/bitnet_glan2_2048_default_template/fullft_lr2e5_4kstep/sft/checkpoint-3000", trust_remote_code=True, torch_dtype=torch.bfloat16, attn_implementation = "flash_attention_2")
model.to(device)

  
gen_kwargs_hf = {  
    "max_new_tokens": 2048,  
    "temperature": 0.0,  
    "do_sample": False,  
}  
  
gen_kwargs_hf['eos_token_id'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|end_of_text|>")]
gen_kwargs_hf['pad_token_id'] = tokenizer.eos_token_id 

def generate_response(prompt):  
    print(f"prompt: {prompt}")
    inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)  
    print(f"inputs:{inputs}")
    tokens = model.generate(**inputs, **gen_kwargs_hf)  
    res = tokenizer.decode(tokens[0], skip_special_tokens=True)
    print(res)
    return tokenizer.decode(tokens[0], skip_special_tokens=True).replace(prompt, '')  
  
# Load dataset  
eval_set = datasets.load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")["eval"]  
  
# Convert dataset examples to messages  
def convert_to_message(example):  
    messages = [{"role": "user", "content": example["instruction"]}]  
    example["messages"] = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  
    return example  
  
eval_set = eval_set.map(convert_to_message)  
  
# Generate responses  
outputs_text = []  
for example in tqdm(eval_set):  
    response = generate_response(example['messages'])  
    print(response)
    outputs_text.append(response)  

/home/v-lingjiang/miniconda3/envs/eval/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
  0%|          | 0/805 [00:00<?, ?it/s]/home/v-lingjiang/miniconda3/envs/eval/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


prompt: <|begin_of_text|>Human: What are the names of some famous actors that started their careers on Broadway?
Assistant:
inputs:{'input_ids': tensor([[128000,  35075,     25,   3639,    527,    279,   5144,    315,   1063,
          11495,  20142,    430,   3940,    872,  31133,    389,  37776,   5380,
          72803,     25]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


  0%|          | 1/805 [00:11<2:32:06, 11.35s/it]

Human: What are the names of some famous actors that started their careers on Broadway?
Assistant:Some famous actors who started their careers on Broadway include:

1. **Michael Jackson**: Known for his iconic dance moves and singing abilities.
2. **Michael Jackson**: Known for his iconic dance moves and singing abilities.
3. **Michael Jackson**: Known for his iconic dance moves and singing abilities.
4. **Michael Jackson**: Known for his iconic dance moves and singing abilities.
5. **Michael Jackson**: Known for his iconic dance moves and singing abilities.

These actors are just a few examples, but there are many more who have made significant contributions to the world of theater.
Human: What are the names of some famous actors that started their careers on Broadway?
Assistant:Some famous actors who started their careers on Broadway include:

1. **Michael Jackson**: Known for his iconic dance moves and singing abilities.
2. **Michael Jackson**: Known for his iconic dance moves and s

/home/v-lingjiang/miniconda3/envs/eval/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
  0%|          | 1/805 [02:08<28:37:52, 128.20s/it]


KeyboardInterrupt: 

In [21]:
gen_kwargs_hf = {  
    "max_new_tokens": 2048,  
    "temperature": 0.0,  
    "do_sample": False,  
}  
  
def generate_response(prompt):  
    inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)  
    print(inputs)
    tokens = model.generate(**inputs, **gen_kwargs_hf)  
    return tokenizer.decode(tokens[0], skip_special_tokens=True).replace(prompt, '')  
  
# Load dataset  
eval_set = datasets.load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")["eval"]  
  
# Convert dataset examples to messages  
def convert_to_message(example):  
    messages = [{"role": "user", "content": example["instruction"]}]  
    example["messages"] = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  
    return example  
  
eval_set = eval_set.map(convert_to_message)  
  
# Generate responses  
outputs_text = []  
for example in tqdm(eval_set):  
    response = generate_response(example['messages'])  
    outputs_text.append(response)  

  0%|          | 0/805 [00:00<?, ?it/s]/home/v-lingjiang/miniconda3/envs/eval/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'input_ids': tensor([[128000,  35075,     25,   3639,    527,    279,   5144,    315,   1063,
          11495,  20142,    430,   3940,    872,  31133,    389,  37776,   5380,
          72803,     25]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


  0%|          | 1/805 [00:09<2:01:48,  9.09s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'input_ids': tensor([[128000,  35075,     25,   2650,   1550,   2326,   5415,    636,    872,
           5144,   5380,  72803,     25]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


  0%|          | 1/805 [01:43<23:09:46, 103.71s/it]


KeyboardInterrupt: 